# Investigate the Quality of Flower102

In [ ]:
#!pip install torch==2.0.0 torchvision scienceplots einops torchinfo randomname seaborn -q

In [ ]:
import torch
from torchvision import datasets

# TODO: replace with package after public on PyPi
import sys

sys.path.append("../")

from src.cleaner.selfclean import PretrainingType, SelfClean

In [ ]:
dataset_splits = []
for split in ["train", "val", "test"]:
    dataset_splits.append(datasets.Flowers102(root="./tmp", download=True, split=split))
dataset = torch.utils.data.ConcatDataset(dataset_splits)

In [ ]:
selfclean = SelfClean(
    auto_cleaning=True,
    plot_distribution=True,
    plot_top_N=7,
)
out_dict = selfclean.run_on_dataset(
    dataset=dataset,
    pretraining_type=PretrainingType.DINO,
    num_workers=24,
    epochs=100,
    batch_size=16,
)

In [ ]:
# TODO: plot dups, irr, lbls individually